In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
import urllib
from bs4 import BeautifulSoup
import time
import csv
import re
import sys
import os
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace

In [2]:
path = 'datasets/alltrails/'

In [3]:
chrome_options = webdriver.chrome.options.Options()
chrome_options.add_argument("--start-maximized")
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')
chrome_path = r'C:\Users\Sumit\Anaconda3\pkgs\python-chromedriver-binary-77.0.3865.40.0-py36_0\Lib\site-packages\chromedriver_binary\chromedriver.exe'

In [4]:
def enter_text(location, url):
    try:
        WebDriverWait(driver, 10).until(ec.presence_of_element_located((By.ID, "search")))
        search_bar = driver.find_element_by_id("search")
        search_bar.clear()
        time.sleep(1)
        search_bar.send_keys(location) # ' '.join(location)
        time.sleep(3)
        search_result = driver.find_elements_by_class_name("place")[0]
        time.sleep(3)
        search_result.click()
        return "Valid"
    except:
        driver.get(url)

def get_elements():
    try:
        WebDriverWait(driver, 10).until(ec.presence_of_element_located((By.CLASS_NAME, 'trail-result-card')))
        time.sleep(2)
        results = driver.find_elements_by_class_name("trail-result-card")
        #print(f"Found {len(results)} trails")
        return results
    except:
        results = None
        return results

def get_trails_write_to_file(results, location, park_abbr, url):
    df_trails = pd.DataFrame()
    
    if results:
        with open(path + str("trails_"+park_abbr+".csv"), "a") as csv_file:
            writer = csv.writer(csv_file)
            try:
                WebDriverWait(driver, 10).until(ec.presence_of_element_located((By.CLASS_NAME, 'result-count')))
                time.sleep(1)
                total_trails = driver.find_element_by_class_name("result-count").text
                print(f"Total trails: {total_trails}")
                for result in results:
                    name = result.find_element_by_class_name("name").text
                    time.sleep(1)
                    difficulty = result.find_element_by_class_name("diff").text
                    time.sleep(1)
                    reviews = result.find_element_by_class_name("review-count").text
                    time.sleep(1)
                    rating = result.find_element_by_class_name("rating").get_attribute("title")
                    time.sleep(1)
                    trail_url = result.find_element_by_tag_name("a").get_attribute("href")
                    time.sleep(1)
                    writer.writerow([location, name, difficulty, reviews, rating, total_trails, trail_url])
                    df_trails = df_trails.append({'location':location, 'park':park_abbr, 'trail':trail_url.split('/')[-1], 'name':name, 'difficulty':difficulty, 'reviews':reviews, 'rating':rating, 'total_trails':total_trails, 'trail_url':trail_url}, ignore_index=True)
                    # print(df_trails)
            except:
                writer.writerow([location,"NA"])
        print(f"Trails from {location} written to file")
    else:
        driver.get(url)
    return df_trails

def login():
    driver.get("https://www.alltrails.com/login?ref=header#")
    email = driver.find_element_by_id("user_email").send_keys("pokeplayer410@outlook.com")
    password = driver.find_element_by_id("user_password").send_keys("insightdatascience")
    login_button = driver.find_element_by_class_name("login")
    login_button.click()

def main(locations_state_df):
    login()
    time.sleep(1)
    
    df_trails = pd.DataFrame()
    
    # For each park
    for index, row in locations_state_df.iloc[::-1].iterrows():
        state = row['state'].strip()
        park_abbr = row['park_abbr'].strip()
        park = row['Park'].strip()
        url = "https://www.alltrails.com/explore/us/"+state+"/"+park_abbr
        
        # First get the trails data for the park
        print(f"Searching trails from: {park_abbr}")
        valid = enter_text(park, url)
        if valid == "Valid":
            print('Valid')
            results = get_elements()
        else:
            results = None
        df_trails = df_trails.append(get_trails_write_to_file(results, park, park_abbr, url), ignore_index=True)
        print(df_trails)
        
        # After main is done, get the reviews data for each trail in the park using the saved file
        
        # Un-comment to get just Yellowstone # Upon wanting to get more park data, comment out the following:
        # break
    print(f"Exiting on {index}")
    return df_trails
    #sys.exit()

In [5]:
# driver = webdriver.Chrome(chrome_path, options=chrome_options)
# df_trail_reviews = get_reviews_write_to_file('yellowstone-national-park', 'Upper Geyser Basin and Old Faithful Observation Point Loop', 'upper-geyser-basin-and-old-faithful-observation-point-loop', 'https://www.alltrails.com/explore/trail/us/wyoming/upper-geyser-basin-and-old-faithful-observation-point-loop')

In [6]:
if True: #if __name__ == "__main__":
    driver = webdriver.Chrome(chrome_path, options=chrome_options)

    locations_df = pd.read_csv('locations.csv', header=None)
    locations = locations_df[1].tolist()
    locations_state_df = pd.read_csv('locations_state.csv')
    df_trails = main(locations_state_df)
    driver.quit()

Searching trails from: yellowstone-national-park
Valid
Total trails: Showing 26 trails
Trails from Yellowstone National Park written to file
   difficulty                   location  \
0        EASY  Yellowstone National Park   
1    MODERATE  Yellowstone National Park   
2        EASY  Yellowstone National Park   
3        EASY  Yellowstone National Park   
4        EASY  Yellowstone National Park   
5        EASY  Yellowstone National Park   
6        EASY  Yellowstone National Park   
7        EASY  Yellowstone National Park   
8        EASY  Yellowstone National Park   
9        EASY  Yellowstone National Park   
10   MODERATE  Yellowstone National Park   
11       EASY  Yellowstone National Park   
12       EASY  Yellowstone National Park   
13   MODERATE  Yellowstone National Park   
14   MODERATE  Yellowstone National Park   
15   MODERATE  Yellowstone National Park   
16   MODERATE  Yellowstone National Park   
17       HARD  Yellowstone National Park   
18   MODERATE  Yellowst

   difficulty                   location  \
0        EASY  Yellowstone National Park   
1    MODERATE  Yellowstone National Park   
2        EASY  Yellowstone National Park   
3        EASY  Yellowstone National Park   
4        EASY  Yellowstone National Park   
5        EASY  Yellowstone National Park   
6        EASY  Yellowstone National Park   
7        EASY  Yellowstone National Park   
8        EASY  Yellowstone National Park   
9        EASY  Yellowstone National Park   
10   MODERATE  Yellowstone National Park   
11       EASY  Yellowstone National Park   
12       EASY  Yellowstone National Park   
13   MODERATE  Yellowstone National Park   
14   MODERATE  Yellowstone National Park   
15   MODERATE  Yellowstone National Park   
16   MODERATE  Yellowstone National Park   
17       HARD  Yellowstone National Park   
18   MODERATE  Yellowstone National Park   
19   MODERATE  Yellowstone National Park   
20   MODERATE  Yellowstone National Park   
21       HARD  Yellowstone Natio

Valid
Total trails: Showing 6 trails
Trails from Fort Laramie National Historic Site written to file
   difficulty                             location  \
0        EASY            Yellowstone National Park   
1    MODERATE            Yellowstone National Park   
2        EASY            Yellowstone National Park   
3        EASY            Yellowstone National Park   
4        EASY            Yellowstone National Park   
5        EASY            Yellowstone National Park   
6        EASY            Yellowstone National Park   
7        EASY            Yellowstone National Park   
8        EASY            Yellowstone National Park   
9        EASY            Yellowstone National Park   
10   MODERATE            Yellowstone National Park   
11       EASY            Yellowstone National Park   
12       EASY            Yellowstone National Park   
13   MODERATE            Yellowstone National Park   
14   MODERATE            Yellowstone National Park   
15   MODERATE            Yellowston

   difficulty                             location  \
0        EASY            Yellowstone National Park   
1    MODERATE            Yellowstone National Park   
2        EASY            Yellowstone National Park   
3        EASY            Yellowstone National Park   
4        EASY            Yellowstone National Park   
5        EASY            Yellowstone National Park   
6        EASY            Yellowstone National Park   
7        EASY            Yellowstone National Park   
8        EASY            Yellowstone National Park   
9        EASY            Yellowstone National Park   
10   MODERATE            Yellowstone National Park   
11       EASY            Yellowstone National Park   
12       EASY            Yellowstone National Park   
13   MODERATE            Yellowstone National Park   
14   MODERATE            Yellowstone National Park   
15   MODERATE            Yellowstone National Park   
16   MODERATE            Yellowstone National Park   
17       HARD            Yel

   difficulty                             location  \
0        EASY            Yellowstone National Park   
1    MODERATE            Yellowstone National Park   
2        EASY            Yellowstone National Park   
3        EASY            Yellowstone National Park   
4        EASY            Yellowstone National Park   
5        EASY            Yellowstone National Park   
6        EASY            Yellowstone National Park   
7        EASY            Yellowstone National Park   
8        EASY            Yellowstone National Park   
9        EASY            Yellowstone National Park   
10   MODERATE            Yellowstone National Park   
11       EASY            Yellowstone National Park   
12       EASY            Yellowstone National Park   
13   MODERATE            Yellowstone National Park   
14   MODERATE            Yellowstone National Park   
15   MODERATE            Yellowstone National Park   
16   MODERATE            Yellowstone National Park   
17       HARD            Yel

...

Valid
Total trails: Showing 13 trails
Trails from Denali National Park written to file
    difficulty                   location  \
0         EASY  Yellowstone National Park   
1     MODERATE  Yellowstone National Park   
2         EASY  Yellowstone National Park   
3         EASY  Yellowstone National Park   
4         EASY  Yellowstone National Park   
..         ...                        ...   
266       EASY       Denali National Park   
267       EASY       Denali National Park   
268   MODERATE       Denali National Park   
269       EASY       Denali National Park   
270       EASY       Denali National Park   

                                                  name  \
0    Upper Geyser Basin and Old Faithful Observatio...   
1    Mystic Falls, Fairy Creek and Little Firehole ...   
2                       Old Faithful Geyser Loop Trail   
3                           Grand Prismatic Hot Spring   
4                               Lone Star Geyser Trail   
..                      

    difficulty                   location  \
0         EASY  Yellowstone National Park   
1     MODERATE  Yellowstone National Park   
2         EASY  Yellowstone National Park   
3         EASY  Yellowstone National Park   
4         EASY  Yellowstone National Park   
..         ...                        ...   
266       EASY       Denali National Park   
267       EASY       Denali National Park   
268   MODERATE       Denali National Park   
269       EASY       Denali National Park   
270       EASY       Denali National Park   

                                                  name  \
0    Upper Geyser Basin and Old Faithful Observatio...   
1    Mystic Falls, Fairy Creek and Little Firehole ...   
2                       Old Faithful Geyser Loop Trail   
3                           Grand Prismatic Hot Spring   
4                               Lone Star Geyser Trail   
..                                                 ...   
266                                   Jonesville Trai

    difficulty                   location  \
0         EASY  Yellowstone National Park   
1     MODERATE  Yellowstone National Park   
2         EASY  Yellowstone National Park   
3         EASY  Yellowstone National Park   
4         EASY  Yellowstone National Park   
..         ...                        ...   
266       EASY       Denali National Park   
267       EASY       Denali National Park   
268   MODERATE       Denali National Park   
269       EASY       Denali National Park   
270       EASY       Denali National Park   

                                                  name  \
0    Upper Geyser Basin and Old Faithful Observatio...   
1    Mystic Falls, Fairy Creek and Little Firehole ...   
2                       Old Faithful Geyser Loop Trail   
3                           Grand Prismatic Hot Spring   
4                               Lone Star Geyser Trail   
..                                                 ...   
266                                   Jonesville Trai

    difficulty                   location  \
0         EASY  Yellowstone National Park   
1     MODERATE  Yellowstone National Park   
2         EASY  Yellowstone National Park   
3         EASY  Yellowstone National Park   
4         EASY  Yellowstone National Park   
..         ...                        ...   
266       EASY       Denali National Park   
267       EASY       Denali National Park   
268   MODERATE       Denali National Park   
269       EASY       Denali National Park   
270       EASY       Denali National Park   

                                                  name  \
0    Upper Geyser Basin and Old Faithful Observatio...   
1    Mystic Falls, Fairy Creek and Little Firehole ...   
2                       Old Faithful Geyser Loop Trail   
3                           Grand Prismatic Hot Spring   
4                               Lone Star Geyser Trail   
..                                                 ...   
266                                   Jonesville Trai

    difficulty                   location  \
0         EASY  Yellowstone National Park   
1     MODERATE  Yellowstone National Park   
2         EASY  Yellowstone National Park   
3         EASY  Yellowstone National Park   
4         EASY  Yellowstone National Park   
..         ...                        ...   
266       EASY       Denali National Park   
267       EASY       Denali National Park   
268   MODERATE       Denali National Park   
269       EASY       Denali National Park   
270       EASY       Denali National Park   

                                                  name  \
0    Upper Geyser Basin and Old Faithful Observatio...   
1    Mystic Falls, Fairy Creek and Little Firehole ...   
2                       Old Faithful Geyser Loop Trail   
3                           Grand Prismatic Hot Spring   
4                               Lone Star Geyser Trail   
..                                                 ...   
266                                   Jonesville Trai

In [7]:
def scrape_trail(browser, hike_url):
    browser.get(hike_url)
    while True:
        try:
            load_more_ratings = WebDriverWait(browser, 2).until(
                ec.visibility_of_element_located((
                By.XPATH,"//div[@id='load_more'] [@class='feed-item load-more'][//a]")))
            load_more_ratings.click()
            time.sleep(2)
        except:
            break
    soup = BeautifulSoup(browser.page_source, "lxml")
    return soup

In [16]:
df_trails['trail'] = df_trails['trail'].str.split('?').str[0]
df_trails['trail_url'] = df_trails['trail_url'].str.split('?').str[0]
df_trails['trail_url'] = df_trails['trail_url'].str.replace('/explore/','/')

In [17]:
driver = webdriver.Chrome(chrome_path, options=chrome_options)
login()
time.sleep(1)

for index, row in df_trails.iterrows():
    park = row['park']
    trail = row['trail']
    trail_url = row['trail_url']
    print(trail_url)
    df_trail_reviews = pd.DataFrame()

    soup = scrape_trail(driver, trail_url)
    #print(soup.getText())

    e_rev = soup.findAll('div', {'itemprop': 'review'})
    print('Reviews: '+str(len(e_rev)))
    for review in e_rev:
        e_rev_date = review.find('meta', {'itemprop': 'datePublished'})
        e_rev_body = review.find('p', {'itemprop': 'reviewBody'})
        rev_date = e_rev_date['content']
        rev_body = e_rev_body.getText()
        # print(rev_date)
        # print(rev_body)
        df_trail_reviews = df_trail_reviews.append({'park':park,'trail':trail,'date':rev_date,'review':rev_body}, ignore_index=True)
    df_trail_reviews.to_csv(path+'reviews_'+park+'_'+trail+'.csv')
driver.quit()

https://www.alltrails.com/trail/us/wyoming/upper-geyser-basin-and-old-faithful-observation-point-loop
Reviews: 312
https://www.alltrails.com/trail/us/wyoming/mystic-falls-fairy-creek-and-little-firehole-loop
Reviews: 313
https://www.alltrails.com/trail/us/wyoming/observation-point-loop-trail
Reviews: 177
https://www.alltrails.com/trail/us/wyoming/grand-prismatic-hot-spring
Reviews: 519
https://www.alltrails.com/trail/us/wyoming/lone-star-geyser-trail
Reviews: 125
https://www.alltrails.com/trail/us/wyoming/mystic-falls--2
Reviews: 76
https://www.alltrails.com/trail/us/wyoming/biscuit-basin-interpretive-trail
Reviews: 108
https://www.alltrails.com/trail/us/wyoming/black-sand-basin-loop-yellowstone-np
Reviews: 61
https://www.alltrails.com/trail/us/wyoming/fairy-falls--4
Reviews: 137
https://www.alltrails.com/trail/us/wyoming/geyser-hill-loop-trail-and-observation-point
Reviews: 48
https://www.alltrails.com/trail/us/wyoming/yellowstone-observation-point
Reviews: 42
https://www.alltrails.co

Reviews: 15
https://www.alltrails.com/trail/us/virginia/lilttle-rocky-row-loop-via-appalachian-trail
Reviews: 24
https://www.alltrails.com/trail/us/virginia/appalachian-trail-to-apple-orchard-falls-to-cornelius-creek-loop
Reviews: 11
https://www.alltrails.com/trail/us/virginia/wildcat-trail
Reviews: 2
https://www.alltrails.com/trail/us/virginia/sprouts-run-wilson-mountain-loop-trail
Reviews: 13
https://www.alltrails.com/trail/us/virginia/petites-gap-sulphur-spring-and-appalachian-trail-loop
Reviews: 2
https://www.alltrails.com/trail/us/virginia/big-rocky-row
Reviews: 8
https://www.alltrails.com/trail/us/virginia/otter-creek-loop
Reviews: 23
https://www.alltrails.com/trail/us/virginia/the-blue-ridge-parkway-james-river-visitor-center-to-roanoke-mountain
Reviews: 5
https://www.alltrails.com/trail/us/virginia/devils-marbleyard-via-belfast-trail
Reviews: 252
https://www.alltrails.com/trail/us/texas/lost-mine-trail
Reviews: 404
https://www.alltrails.com/trail/us/texas/south-rim-trail-boot-s

Reviews: 4
https://www.alltrails.com/trail/us/wyoming/delacy-creek-trail-to-shoshone-lake
Reviews: 13
https://www.alltrails.com/trail/us/wyoming/divide-mountain-trail
Reviews: 6
https://www.alltrails.com/trail/us/wyoming/upper-geyser-basin-and-old-faithful-observation-point-loop
Reviews: 312
https://www.alltrails.com/trail/us/colorado/far-view-sites-complex
Reviews: 26
https://www.alltrails.com/trail/us/colorado/mesa-verde-point-lookout-trail
Reviews: 62
https://www.alltrails.com/trail/us/colorado/park-point-overlook-trail
Reviews: 24
https://www.alltrails.com/trail/us/colorado/long-house-trail
Reviews: 40
https://www.alltrails.com/trail/us/colorado/step-house-trail
Reviews: 48
https://www.alltrails.com/trail/us/colorado/knife-edge-trail
Reviews: 29
https://www.alltrails.com/trail/us/colorado/prater-ridge-trail
Reviews: 31
https://www.alltrails.com/trail/us/colorado/badger-house-community-trail
Reviews: 14
https://www.alltrails.com/trail/us/colorado/farming-terraces-trail
Reviews: 19
h

In [ ]:
# trail_url = 'https://www.alltrails.com/trail/us/wyoming/upper-geyser-basin-and-old-faithful-observation-point-loop'
# driver = webdriver.Chrome(chrome_path, options=chrome_options)
# soup = scrape_trail(driver, trail_url)
# # print(soup.getText())

# e_rev = soup.findAll('div', {'itemprop': 'review'})
# for review in e_rev:
#     # print(review.getText())
#     e_rev_date = review.find('meta', {'itemprop': 'datePublished'})
#     e_rev_body = review.find('p', {'itemprop': 'reviewBody'})
#     rev_date = e_rev_date['content']
#     rev_body = e_rev_body.getText()
#     # print(rev_date)
#     # print(rev_body)

In [ ]:
df_trails

In [4]:
locations

['Abraham Lincoln Birthplace National Historical Park',
 'Acadia National Park',
 'Adams National Historical Park',
 'African American Civil War Memorial',
 'African Burial Ground National Monument',
 'Agate Fossil Beds National Monument',
 'Ala Kahakai National Historic Trail',
 'Alagnak Wild River',
 'Alcatraz Island',
 'Aleutian World War II National Historic Area',
 'Alibates Flint Quarries National Monument',
 'Allegheny Portage Railroad National Historic Site',
 'American Memorial Park',
 'Amistad National Recreation Area',
 'Anacostia Park',
 'Andersonville National Historic Site',
 'Andrew Johnson National Historic Site',
 'Aniakchak National Monument and Preserve',
 'Antietam National Battlefield',
 'Antietam National Cemetery',
 'Apostle Islands National Lakeshore',
 'Appalachian National Scenic Trail',
 'Appomattox Court House National Historical Park',
 'Arches National Park',
 'Arkansas Post National Memorial',
 'Arlington House, The Robert E. Lee Memorial',
 'Assateague I

In [19]:
import pickle
outfile=open('AllTrailsScraper_notebook_env.db','wb')
pickle.dump(df_trails,outfile)
outfile.close()

In [13]:
# print(soup.prettify())